In [1]:
%whos
# %reset

Interactive namespace is empty.


In [3]:
from bs4 import BeautifulSoup
import requests
import copy
import re
import pickle
from collections import defaultdict
import sys
import matplotlib
import matplotlib.pyplot as plt
import time
sys.setrecursionlimit(40000) # to enable pickle to handle the dictionary load

In [9]:
# just the first time around
# final_dict = {}

In [2]:
# GET and prettify via Beautiful Soup
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} # This is chrome, you can set whatever browser you like
response = requests.get("https://www.glassdoor.com/Reviews/new-york-city-reviews-SRCH_IL.0,13_IM615.htm", headers=headers)
print(response.status_code)

200


In [3]:
co_list = []
soup = BeautifulSoup(response.content, 'html.parser') #Pass the page contents to beautiful soup for parsing
# soup.prettify # test-print
cos = soup.findAll('a', {'class':'tightAll h2'})

for c in cos:
    co_list.append(c.get_text())
print(co_list)

[' J.P. Morgan ', ' IBM ', ' Citi ', " Macy's ", ' Verizon ', ' Morgan Stanley ', ' Goldman Sachs ', ' PwC ', ' EY ', ' Deloitte ']


In [4]:
reviews = soup.findAll('a', {'class':'sqLogoLink'})
revs_list = []
for r in reviews:
    revs_list.append('https://www.glassdoor.com'+r.get('href', '\n'))
print(revs_list)

['https://www.glassdoor.com/Overview/Working-at-J-P-Morgan-EI_IE145.11,21.htm', 'https://www.glassdoor.com/Overview/Working-at-IBM-EI_IE354.11,14.htm', 'https://www.glassdoor.com/citi', 'https://www.glassdoor.com/Overview/Working-at-Macy-s-EI_IE1079.11,17.htm', 'https://www.glassdoor.com/Overview/Working-at-Verizon-EI_IE89.11,18.htm', 'https://www.glassdoor.com/Overview/Working-at-Morgan-Stanley-EI_IE2282.11,25.htm', 'https://www.glassdoor.com/Overview/Working-at-Goldman-Sachs-EI_IE2800.11,24.htm', 'https://www.glassdoor.com/Overview/Working-at-PwC-EI_IE8450.11,14.htm', 'https://www.glassdoor.com/Overview/Working-at-EY-EI_IE2784.11,13.htm', 'https://www.glassdoor.com/Overview/Working-at-Deloitte-EI_IE2763.11,19.htm']


In [5]:
main_dict = {}
for i in range(10):
    main_dict[co_list[i]] = revs_list[i]
print(main_dict)

{' J.P. Morgan ': 'https://www.glassdoor.com/Overview/Working-at-J-P-Morgan-EI_IE145.11,21.htm', ' IBM ': 'https://www.glassdoor.com/Overview/Working-at-IBM-EI_IE354.11,14.htm', ' Citi ': 'https://www.glassdoor.com/citi', " Macy's ": 'https://www.glassdoor.com/Overview/Working-at-Macy-s-EI_IE1079.11,17.htm', ' Verizon ': 'https://www.glassdoor.com/Overview/Working-at-Verizon-EI_IE89.11,18.htm', ' Morgan Stanley ': 'https://www.glassdoor.com/Overview/Working-at-Morgan-Stanley-EI_IE2282.11,25.htm', ' Goldman Sachs ': 'https://www.glassdoor.com/Overview/Working-at-Goldman-Sachs-EI_IE2800.11,24.htm', ' PwC ': 'https://www.glassdoor.com/Overview/Working-at-PwC-EI_IE8450.11,14.htm', ' EY ': 'https://www.glassdoor.com/Overview/Working-at-EY-EI_IE2784.11,13.htm', ' Deloitte ': 'https://www.glassdoor.com/Overview/Working-at-Deloitte-EI_IE2763.11,19.htm'}


In [6]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} # This is chrome, you can set whatever browser you like
response_jpm = requests.get("https://www.glassdoor.com/Overview/Working-at-J-P-Morgan-EI_IE145.11,21.htm", headers=headers)
info_list = []
soup = BeautifulSoup(response_jpm.content, 'html.parser') #Pass the page contents to beautiful soup for parsing
# soup.prettify # test-print

In [33]:
info = soup.findAll('div', {'class':'info flexbox row col-hh'})
labels_list = []
for i in info:
     labels_t = i.findAll('label')
for l in labels_t:
    labels_list.append(l.get_text())
labels_list

['Website', 'Headquarters', 'Size', 'Founded', 'Type', 'Industry', 'Revenue']

In [42]:
info = soup.findAll('span', {'class':'value'})

for i in info:
    info_list.append(i.get_text())
print(info_list)

['www.jpmorgan.com', 'New York, NY', '10000+ employees', ' 1799', ' Company - Public (JPM) ', ' Investment Banking & Asset Management', ' $10+ billion (USD) per year']


In [4]:
# Your code here; use a function or loop to retrieve all the results from your original request
def co_call(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} # This is chrome, you can set whatever browser you like
    response = requests.get(url, headers=headers)
    print(url, response.status_code)
    
    co_list = []
    revs_list = []
    soup = BeautifulSoup(response.content, 'html.parser') #Pass the page contents to beautiful soup for parsing

    cos = soup.findAll('a', {'class':'tightAll h2'})
    for c in cos:
        co_list.append(c.get_text())

    reviews = soup.findAll('a', {'class':'sqLogoLink'})
    for r in reviews:
        revs_list.append('https://www.glassdoor.com'+r.get('href', '\n'))
    
    main_dict = {}
    for i in range(10):
        # print(i, co_list[i], revs_list[i])
        main_dict[co_list[i]] = revs_list[i]
    return main_dict

####################
last_page = 50
cur_page = 46
results = {}
while cur_page <= last_page:
    if cur_page == 1:
        url = 'https://www.glassdoor.com/Reviews/new-york-city-reviews-SRCH_IL.0,13_IM615.htm'
    else:
        url = 'https://www.glassdoor.com/Reviews/new-york-city-reviews-SRCH_IL.0,13_IM615_IP'+str(cur_page)+'.htm'
    results.update(g_call(url))
    time.sleep(1) #Wait a second
    cur_page += 1
print(results)

NameError: name 'g_call' is not defined

In [50]:
# Your code here; use a function or loop to retrieve all the results from your original request

def g_call(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} # This is chrome, you can set whatever browser you like
    response = requests.get(url, headers=headers)
    print(url, response.status_code)
    
    co_list = []
    revs_list = []
    val_dict = {}
    soup = BeautifulSoup(response.content, 'html.parser') #Pass the page contents to beautiful soup for parsing

    cos = soup.findAll('a', {'class':'tightAll h2'})
    for c in cos:
        co_list.append(c.get_text())

    reviews = soup.findAll('a', {'class':'sqLogoLink'})
    for r in reviews:
        revs_list.append('https://www.glassdoor.com'+r.get('href', '\n'))
    
    main_dict = {}
    for i in range(10):
        # print(i, co_list[i], revs_list[i])
        main_dict[co_list[i]] = {'link':revs_list[i]}
    return main_dict

####################
last_page = 50
cur_page = 46
results = {}
while cur_page <= last_page:
    if cur_page == 1:
        url = 'https://www.glassdoor.com/Reviews/new-york-city-reviews-SRCH_IL.0,13_IM615.htm'
    else:
        url = 'https://www.glassdoor.com/Reviews/new-york-city-reviews-SRCH_IL.0,13_IM615_IP'+str(cur_page)+'.htm'
    results.update(g_call(url))
    time.sleep(1) #Wait a second
    cur_page += 1
print(len(results))
# print(results)

https://www.glassdoor.com/Reviews/new-york-city-reviews-SRCH_IL.0,13_IM615_IP46.htm 200
https://www.glassdoor.com/Reviews/new-york-city-reviews-SRCH_IL.0,13_IM615_IP47.htm 200
https://www.glassdoor.com/Reviews/new-york-city-reviews-SRCH_IL.0,13_IM615_IP48.htm 200
https://www.glassdoor.com/Reviews/new-york-city-reviews-SRCH_IL.0,13_IM615_IP49.htm 200
https://www.glassdoor.com/Reviews/new-york-city-reviews-SRCH_IL.0,13_IM615_IP50.htm 200
50


In [51]:
final_dict.update(results)
print(len(final_dict.keys()))
print(final_dict.keys())

500
dict_keys([' J.P. Morgan ', ' IBM ', ' Citi ', " Macy's ", ' Verizon ', ' Morgan Stanley ', ' Goldman Sachs ', ' PwC ', ' EY ', ' Deloitte ', ' Cognizant Technology Solutions ', ' Bloomberg L.P. ', ' Bank of America ', ' American Express ', ' Columbia University ', ' AIG ', ' New York City Department of Education ', ' NYU (New York University) ', ' KPMG ', ' Viacom ', ' Rutgers University ', ' Johnson & Johnson ', ' UBS ', ' Target ', ' Credit Suisse ', ' Thomson Reuters ', ' Starbucks ', ' Barclays ', ' AT&T ', ' BNY Mellon ', ' The Home Depot ', ' Mount Sinai Health System ', ' CVS Health ', ' Deutsche Bank ', ' Apple ', ' Accenture ', ' NBCUniversal ', ' Northwell Health ', ' NewYork-Presbyterian Hospital ', ' Memorial Sloan Kettering ', ' UPS ', ' Best Buy ', ' Prudential ', ' NYU Langone Health ', ' ADP ', " Bloomingdale's ", ' Pfizer ', ' New York Life ', ' Merck ', ' Chase ', ' Ralph Lauren ', ' TD ', ' Toys "R" Us ', " McDonald's ", ' MetLife ', ' Staples ', ' Shoprite Supe

In [113]:
# for k in final_dict:
#     print(final_dict[k]['link'])
len(final_dict)

500

In [52]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} # This is chrome, you can set whatever browser you like

for key in final_dict:
    # request to company link
    response = requests.get(final_dict[key]['link'], headers=headers)
    print(final_dict[key]['link'], response.status_code)
    soup = BeautifulSoup(response.content, 'html.parser') #Pass the page contents to beautiful soup for parsing
    
    # pick up company's labels
    info = soup.findAll('div', {'class':'info flexbox row col-hh'})
    labels_list = []
    for i in info:
        labels_t = i.findAll('label')
    for l in labels_t:
        labels_list.append(l.get_text())
    # pick up company's label values
    info_vals = soup.findAll('span', {'class':'value'})
    values_list = []
    for i in info_vals:
        values_list.append(i.get_text())
    if len(labels_list) != len(values_list):
        print('Error')
    else:
        for i in range(len(labels_list)):
            final_dict[key][labels_list[i]] = values_list[i]

https://www.glassdoor.com/Overview/Working-at-J-P-Morgan-EI_IE145.11,21.htm 200
https://www.glassdoor.com/Overview/Working-at-IBM-EI_IE354.11,14.htm 200
https://www.glassdoor.com/citi 200
https://www.glassdoor.com/Overview/Working-at-Macy-s-EI_IE1079.11,17.htm 200
https://www.glassdoor.com/Overview/Working-at-Verizon-EI_IE89.11,18.htm 200
https://www.glassdoor.com/Overview/Working-at-Morgan-Stanley-EI_IE2282.11,25.htm 200
https://www.glassdoor.com/Overview/Working-at-Goldman-Sachs-EI_IE2800.11,24.htm 200
https://www.glassdoor.com/Overview/Working-at-PwC-EI_IE8450.11,14.htm 200
https://www.glassdoor.com/Overview/Working-at-EY-EI_IE2784.11,13.htm 200
https://www.glassdoor.com/Overview/Working-at-Deloitte-EI_IE2763.11,19.htm 200
https://www.glassdoor.com/Overview/Working-at-Cognizant-Technology-Solutions-EI_IE8014.11,41.htm 200
https://www.glassdoor.com/Overview/Working-at-Bloomberg-L-P-EI_IE3096.11,24.htm 200
https://www.glassdoor.com/Overview/Working-at-Bank-of-America-EI_IE8874.11,26.h

https://www.glassdoor.com/Overview/Working-at-Rite-Aid-EI_IE565.11,19.htm 200
https://www.glassdoor.com/Overview/Working-at-Saks-Fifth-Avenue-EI_IE2963.11,28.htm 200
https://www.glassdoor.com/Overview/Working-at-CUNY-City-University-of-New-York-EI_IE3726.11,43.htm 200
https://www.glassdoor.com/Overview/Working-at-J-Crew-EI_IE2848.11,17.htm 200
https://www.glassdoor.com/Overview/Working-at-Unilever-EI_IE3513.11,19.htm 200
https://www.glassdoor.com/Overview/Working-at-Avon-EI_IE67.11,15.htm 200
https://www.glassdoor.com/Overview/Working-at-Mastercard-EI_IE3677.11,21.htm 200
https://www.glassdoor.com/Overview/Working-at-Tiffany-and-Co-EI_IE663.11,25.htm 200
https://www.glassdoor.com/Overview/Working-at-Town-Sports-International-Holdings-EI_IE8690.11,45.htm 200
https://www.glassdoor.com/Overview/Working-at-Robert-Half-EI_IE1838.11,22.htm 200
https://www.glassdoor.com/Overview/Working-at-YMCA-EI_IE3046.11,15.htm 200
https://www.glassdoor.com/Overview/Working-at-Enterprise-EI_IE2783.11,21.ht

https://www.glassdoor.com/Overview/Working-at-Modell-s-Sporting-Goods-EI_IE4382.11,34.htm 200
https://www.glassdoor.com/Overview/Working-at-Stevens-Institute-of-Technology-EI_IE24409.11,42.htm 200
https://www.glassdoor.com/Overview/Working-at-Nomura-Holdings-EI_IE3475.11,26.htm 200
https://www.glassdoor.com/Overview/Working-at-Benjamin-Moore-EI_IE3089.11,25.htm 200
https://www.glassdoor.com/Overview/Working-at-PSEG-EI_IE16719.11,15.htm 200
https://www.glassdoor.com/Overview/Working-at-Major-League-Baseball-MLB-EI_IE2883.11,36.htm 200
https://www.glassdoor.com/Overview/Working-at-Hollister-Co-EI_IE466037.11,23.htm 200
https://www.glassdoor.com/Overview/Working-at-Stryker-EI_IE1918.11,18.htm 200
https://www.glassdoor.com/Overview/Working-at-Zara-EI_IE17544.11,15.htm 200
https://www.glassdoor.com/Overview/Working-at-Quest-Diagnostics-EI_IE6655.11,28.htm 200
https://www.glassdoor.com/Overview/Working-at-WeWork-EI_IE661275.11,17.htm 200
https://www.glassdoor.com/Overview/Working-at-City-of-

https://www.glassdoor.com/Overview/Working-at-BBDO-EI_IE14065.11,15.htm 200
https://www.glassdoor.com/Overview/Working-at-MediaMath-EI_IE346865.11,20.htm 200
https://www.glassdoor.com/Overview/Working-at-Bain-and-Company-EI_IE3752.11,27.htm 200
https://www.glassdoor.com/Overview/Working-at-Achievement-First-EI_IE297029.11,28.htm 200
https://www.glassdoor.com/Overview/Working-at-NYSE-EI_IE3307.11,15.htm 200
https://www.glassdoor.com/Overview/Working-at-Amplify-EI_IE689638.11,18.htm 200
https://www.glassdoor.com/Overview/Working-at-Dollar-Tree-EI_IE4033.11,22.htm 200
https://www.glassdoor.com/Overview/Working-at-XO-Group-EI_IE9843.11,19.htm 200
https://www.glassdoor.com/Overview/Working-at-A-E-Networks-EI_IE5485.11,23.htm 200
https://www.glassdoor.com/Overview/Working-at-Ladders-EI_IE39451.11,18.htm 200
https://www.glassdoor.com/Overview/Working-at-Aramark-EI_IE2716.11,18.htm 200
https://www.glassdoor.com/Overview/Working-at-Publicis-Sapient-EI_IE1646026.11,27.htm 200
https://www.glassdo

https://www.glassdoor.com/Overview/Working-at-Roche-EI_IE3480.11,16.htm 200
https://www.glassdoor.com/Overview/Working-at-Grant-Thornton-EI_IE100134.11,25.htm 200
https://www.glassdoor.com/Overview/Working-at-iHeartMedia-EI_IE354635.11,22.htm 200
https://www.glassdoor.com/Overview/Working-at-J-Walter-Thompson-Worldwide-EI_IE13666.11,38.htm 200
https://www.glassdoor.com/Overview/Working-at-Bank-Of-Tokyo-Mitsubishi-UFJ-EI_IE240039.11,39.htm 200
https://www.glassdoor.com/Overview/Working-at-Ipreo-EI_IE12636.11,16.htm 200
https://www.glassdoor.com/Overview/Working-at-Siemens-EI_IE3510.11,18.htm 200
https://www.glassdoor.com/spotify 200
https://www.glassdoor.com/Overview/Working-at-Fidelis-Care-EI_IE27742.11,23.htm 200
https://www.glassdoor.com/Overview/Working-at-The-NPD-Group-EI_IE6937.11,24.htm 200
https://www.glassdoor.com/Overview/Working-at-VaynerMedia-EI_IE482441.11,22.htm 200
https://www.glassdoor.com/Overview/Working-at-Rockefeller-University-EI_IE5419.11,33.htm 200
https://www.gla

https://www.glassdoor.com/Overview/Working-at-Subway-EI_IE2994.11,17.htm 200
https://www.glassdoor.com/Overview/Working-at-DICK-S-Sporting-Goods-EI_IE8666.11,32.htm 200
https://www.glassdoor.com/Overview/Working-at-American-Airlines-EI_IE8.11,28.htm 200
https://www.glassdoor.com/Overview/Working-at-EXL-Service-EI_IE37901.11,22.htm 200
https://www.glassdoor.com/Overview/Working-at-Allergan-EI_IE27.11,19.htm 200
https://www.glassdoor.com/Overview/Working-at-NBA-EI_IE2908.11,14.htm 200
https://www.glassdoor.com/Overview/Working-at-Simon-and-Schuster-EI_IE4148.11,29.htm 200
https://www.glassdoor.com/Overview/Working-at-AHRC-New-York-City-EI_IE456300.11,29.htm 200
https://www.glassdoor.com/sap 200
https://www.glassdoor.com/Overview/Working-at-IKEA-EI_IE3957.11,15.htm 200
https://www.glassdoor.com/Overview/Working-at-GNC-EI_IE2270.11,14.htm 200
https://www.glassdoor.com/Overview/Working-at-Huntington-Learning-Centers-EI_IE7937.11,38.htm 200
https://www.glassdoor.com/Overview/Working-at-Bluew

In [53]:
len(final_dict)

500

In [57]:
# Save & Store File (Pickle)
filename = "final_dict"
outfile = open(filename, 'wb')
pickle.dump(final_dict, outfile)
outfile.close()

In [58]:
outfile.close()

In [ ]:
# Load the dictionary back from the pickle file.
new_dict = pickle.load( open( "final_dict", "rb" ) )

In [ ]:
# Retrieve File (Un-pickle)
# infile = open(filename,'rb')
# new_dict = pickle.load(infile)
infile.close()
# confirm it's loaded accurately
print(new_dict)
print(new_dict==final_dict)
print(type(new_dict))

In [ ]:
https://www.glassdoor.com/Reviews/new-york-city-reviews-SRCH_IL.0,13_IM615.htm
https://www.glassdoor.com/Reviews/new-york-city-reviews-SRCH_IL.0,13_IM615_IP2.htm
https://www.glassdoor.com/Reviews/new-york-city-reviews-SRCH_IL.0,13_IM615_IP3.htm
<a href="/Overview/Working-at-J-P-Morgan-EI_IE145.11,21.htm" data-ajax="true" class="sqLogoLink"><span class="sqLogo tighten lgSqLogo logoOverlay"><img src="https://media.glassdoor.com/sql/145/j-p-morgan-squarelogo-1479932535271.png" class="" alt="J.P. Morgan Logo" title=""></span></a>
<div class="info flexbox row col-hh"><div class="infoEntity"><label>Website</label><span class="value website"><a class="link" href="http://www.jpmorgan.com" target="_blank" rel="nofollow noreferrer">www.jpmorgan.com</a></span></div><div class="infoEntity"><label>Headquarters</label><span class="value">New York, NY</span></div><div class="infoEntity"><label>Size</label><span class="value">10000+ employees</span></div><div class="infoEntity"><label>Founded</label><span class="value"> 1799</span></div><div class="infoEntity"><label>Type</label><span class="value"> Company - Public (JPM) </span></div><div class="infoEntity"><label>Industry</label><span class="value"> Investment Banking &amp; Asset Management</span></div><div class="infoEntity"><label>Revenue</label><span class="value"> $10+ billion (USD) per year</span></div></div>


In [ ]:
# GET and prettify via Beautiful Soup
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} # This is chrome, you can set whatever browser you like
response = requests.get("https://www.glassdoor.com/Reviews/new-york-city-reviews-SRCH_IL.0,13_IM615.htm", headers=headers)
print(response.status_code)